In [ ]:
# Quick hack to put us in the icenet-pipeline folder,
# assuming it was created as per 01.cli_demonstration.ipynb
import os
if os.path.exists("02.data_and_forecasts.ipynb"):
    os.chdir("../notebook-pipeline")
print("Running in {}".format(os.getcwd()))

%matplotlib inline

# IceNet Data and Forecast Products

## Context

### Purpose
The IceNet library provides the ability to download, process, train and predict from end to end via a set of command-line interfaces (CLI).

Using this notebook one can understand the various data sources, intermediaries and products that arise from the [CLI demonstrator notebook](01.cli_demonstration.ipynb) activities.

### Modelling approach
This modelling approach allows users to immediately utilise the library for producing sea ice concentraion forecasts.

### Highlights
The key features of an end to end run are: 
* Setup: _this was concerned with setting up the conda environment, which remains the same_
* [Download](#Download): we explore the source data downloaded under the `/data/` folder, reusable across multiple environments
* [Process](#Process): we explore the preprocessing outputs in the `/processed/` folder, which can be fed to models directly or further processed into IceNet datasets
* [Train](#Train): we explore the outputs from the training process and ensemble runs, stored within `/results/networks/`
* [Predict](#Predict): we explore the output from the prediction process and ensemble runs, stored within `/results/predict/`

_This follows the same structure as the CLI demonstration notebook so that it's easy to follow step-by-step..._

### Contributions
#### Notebook
James Byrne (author)

__Please raise issues [in this repository](https://github.com/icenet-ai/icenet-notebooks/issues) to suggest updates to this notebook!__ 

Contact me at _jambyr \<at\> bas.ac.uk_ for anything else...

#### Modelling codebase
James Byrne (code author), Tom Andersson (science author)

#### Modelling publications
Andersson, T.R., Hosking, J.S., Pérez-Ortiz, M. et al. Seasonal Arctic sea ice forecasting with probabilistic deep learning. Nat Commun 12, 5124 (2021). https://doi.org/10.1038/s41467-021-25257-4

#### Involved organisations
The Alan Turing Institute and British Antarctic Survey

## Setup

For the purposes of python analysis we use and provide the following header libraries which are heavily utilised within the IceNet project and the pipeline.

In [ ]:
import glob, json, os
import datetime as dt
import pandas as pd, xarray as xr, matplotlib.pyplot as plt
from IPython.display import HTML

## Download

Downloading data using the icenet_data commands produces a dataset specific input data storage directory called `/data` whose source data can be reused across normalisation (`icenet_process*`) and dataset production (`icenet_dataset*`) runs.

In [ ]:
os.listdir("data")

The structure of these directories (aside from masks) have consistent layouts:

In [ ]:
glob.glob("data/**/2020.nc", recursive=True)

With masks being the only caveat. However, masks can be interacted with purely through the `Masks` class from `icenet2.data.sic.mask`.

In [ ]:
glob.glob("data/masks/**/*.*", recursive=True)

Note that the `siconca` variable files are source files for generating the masks that are not actually used after initial mask creation.

### Producing data source videos

One of the easiest ways to inspect source data is to use the `icenet_video_data` command, which will output to `/plot/` in the run directory video(s) corresponding to the selected dataset, hemisphere, variable and year, or all of these items if run with: 

In [ ]:
!icenet_video_data --years 2020 era5,osisaf

Optionally, we can display the videos right here in the notebook:

In [ ]:
from IPython.display import Video
Video("plot/data_era5_south_tas.mp4", embed=True, width=750)

The equivalent can be run for any dataset under `/data/` with this command. The command also exposes numerous options allowing you to refine via the aforementioned items. Consult `-h` for more information.

### Data storage structure

The reason for this structure is that it applies consistency no matter how many different implementations of data downloaders and data processors are in place. The IceNet library (see next notebook) inherits this structure per-implementation from a common set of parents. Aside from making programmitically overriding and implementing new functionality easier, this __consistency means that plotting and analysis of the data stores becomes trivial__, aiding both research and production analysis of data all the way through the pipeline, however it's implemented.

For example, looking at the southern hemisphere sea surface temperature in our example source datastore for the 1st January 2020.

In [ ]:
xr.plot.contourf(xr.open_dataset("data/era5/south/tas/2020.nc").isel(time=0).tas, levels=50)

Against the sea ice for the same day is super intuitive to derive from the filesystem naming, by simply changing the respective dataset and variable.

In [ ]:
xr.plot.contourf(xr.open_dataset("data/osisaf/south/siconca/2020.nc").isel(time=0).ice_conc, levels=50)

Similarly, switching to the normalised versions (change to `/processed/`) and/or checking other datasets and/or globbing for datasets that can be opened with [`xarray.open_mfdataset`](https://xarray.pydata.org/en/stable/generated/xarray.open_mfdataset.html) is equally trivial thanks to this consistency.

In [ ]:
xr.plot.contourf(xr.open_dataarray("processed/notebook_data/osisaf/south/siconca/siconca_abs.nc").isel(time=0), levels=50)

As we'll see later, extending the library to incorporate additional source data simply adds further entries to the `/data` directory, with the implementations ultimately following a consistent approach to storing the data to make more complex analysis trivial.

#### Dask example

For example, needing to inspect the maximum values for sea surface temperature across all 1990's and 2000's data using dask, xarray and glob

```
from dask.distributed import Client
dfs = glob.glob("data/era5/south/tos/19*.nc") + glob.glob("data/era5/south/tos/20*.nc")
client = Client()
ds = xr.open_mfdataset(dfs, combine="nested", concat_dim="time", parallel=True)
a = ds.groupby("time.year").max("time").max(("yc", "xc"))
m = a.compute()
for y in m.year.values:
    print("{} {:.2f}".format(y, float(m.sel(year=y))))
```

## Process

There is a level of consistency (notwithstanding the masks and [this currently open issue for clarity](https://github.com/icenet-ai/icenet/issues/3)) when looking at the processed files. The key here is that the data is normalised as appropriate via the use of processors defined in `icenet.data.processors`. 

The associated normalisation/climatological parameters are then stored in an _identified_ processed data folder, with a corresponding `loader.name.json` configuration file which takes the form: 

In [ ]:
!cat loader.notebook_data.json

This configuration tracks the various outputs from the normalisation process for each processor used, ensuring that no source data is normalised or modified and can thus be reused across different date ranges easily. At the same time, the caching of this data allows for two additional benefits aside from the obvious separation of preprocessing from data usage. Firstly, the prenormalisation allows uncached datasets to be used directly for predictions and training if desirable and, secondly, it allows multiple datasets to be derived from a single normalisation run which with larger datasets can take a reasonable amount of compute to achieve.

Finally, storing this in a machine and human readable format makes datasets relatively easy to inspect via the loader configuration when you forget what commands you preprocessed with!

__Note that the deeper elements of the configuration file aren't shown for brevity...__

### Producing preprocessed data videos

As with data sources, normalised data is easily visualized at the command line by running the `icenet_video_data` command, where `-p` specifies path to the directory under `/processed/`:

In [ ]:
!icenet_video_data -p processed/notebook_data era5,osisaf

In [ ]:
from IPython.display import Video
Video("plot/processed_notebook_data_era5_south_uas.mp4", embed=True, width=750)

## Train

Training outputs are stored in their respectively named folders using the name provided from the commands `icenet_train` or `run_train_ensemble` respectively. The contents of these folders differ slightly, based simply on the number of runs executed.

In [ ]:
os.listdir("results/networks")

The output from the single run provides two appropriately named files, the JSON of the history object from the run and the weights of the network after training.

In [ ]:
glob.glob("results/networks/notebook_testrun/**", recursive=True)

Similarly, the ensemble outputs the same files across each individual run, distinguished by the seed.

In [ ]:
glob.glob("results/networks/notebook_ensemble/**/*.h5", recursive=True) + glob.glob("results/networks/notebook_ensemble/**/*.json", recursive=True)

In [ ]:
with open("results/networks/notebook_ensemble/notebook_ensemble_45_history.json") as fh:
    history = json.load(fh)

# Quick and dirty visualisation
fig = plt.figure(figsize=(int(len(history.keys())/2) * 3, 15))
fig.tight_layout()
plt.subplots_adjust(hspace=0.5)

for i, k in enumerate(history.keys()):
    ax = fig.add_subplot(int(len(history.keys())/2) + len(history.keys())%2, 2, i+1)

    ax.plot(history[k].keys(), history[k].values())
    ax.set_title(k)

fig.show()

It's important to note that other outputs result, depending on arguments provided and configuration, from `icenet_train` and `run_icenet_ensemble`. Both tensorboard and WandB outputs can be provided and, in the case of the latter, automatically uploaded when using the CLI tools. The output files are also retained in the run folders, with the latter ensemble members being stored in the `/ensemble/<ensembleName>/<ensembleName>-<runNumber>` directory:

In [ ]:
os.listdir("ensemble/notebook_ensemble/notebook_ensemble-0")

Depending on the mechanism (in this case SLURM on our HPC) used to run the ensemble, the log files provide the full output from the run:

In [ ]:
!cat ensemble/notebook_ensemble/notebook_ensemble-0/train.5777508.node021.42.out

Of course in the case of individual runs using `icenet_train` this output will have been presented to stdout/stderr (the screen) whilst running...

## Predict

The training runs provide insight into the training of the network (using the CLI in a very simple fashion) but the predictions produce actual forecast output to be studied or utilised. This can be evaluated as befits the user in comparison to the original SIC data or any other source to validate the forecasts. **Obviously with our notebook network these won't be accurate** but in real use you'd be expecting to produce comparable forecasts with the physics based data used to train the network.

All outputs for prediction runs using either `icenet_predict` or `run_predict_ensemble` are stored in `results/predict`. For our notebook test and ensemble runs we can see the folder structure is similar to the training outputs, with folders named using the identifier:

In [ ]:
glob.glob("results/predict/**")

In [ ]:
os.listdir("results/predict/example_south_ensemble_forecast")

In [ ]:
os.listdir("results/predict/example_south_ensemble_forecast/notebook_ensemble.45")

The prediction per-network folder contains output from the network for the inputs provided, with the dates being the forecast start date. These files correspond to the *original dates we provided in the first notebook*. 

In [ ]:
!cat testdates.csv

In the `loader` are also contained `outputs` and `weights`. The former "outputs" are the generated outputs for the network that would've been used for training. Remember, that to produce a forecast we use an input linear trend SIC channel set, alongside the atmospheric and other data channels. The weights is the collection of sample weights associated to the input set. These are really provided for debugging purposes, but may also be of some analytical interest in some cases. 

With the run of `icenet_output` as shown in the first notebook, these numpy-saved outputs are converted to a CF-compliant NetCDF that can be forwarded on or usefully analysed. 

### Plotting a forecast

The following snippet uses one of these NetCDF's for plotting a forecast:

In [ ]:
from icenet.plotting.video import xarray_to_video as xvid
from icenet.data.sic.mask import Masks

ds = xr.open_dataset("results/predict/example_south_ensemble_forecast.nc")
land_mask = Masks(south=True, north=False).get_land_mask()
ds.info()

In [ ]:
forecast_date = ds.time.values[0]
print(forecast_date)

In [ ]:
fc = ds.sic_mean.isel(time=0).drop_vars("time").rename(dict(leadtime="time"))
fc['time'] = [pd.to_datetime(forecast_date) \
              + dt.timedelta(days=int(e)) for e in fc.time.values]

anim = xvid(fc, 15, figsize=4, mask=land_mask)
HTML(anim.to_jshtml())

Obviously, with the size and training of the notebook dataset this is not a particularly useful forecast but the ease of generating a visual is hopefully apparent. Also, this segways nicely into the next notebook as we've started to introduce leverages methods and classes directly from the `icenet` API itself. 

## Summary

In this notebook we've explored the data assets that are stored based on the end to end run of the pipeline, hopefully enough to make the user aware of the structuring of the run directory when using either single-run or ensemble-based mechanisms of execution. Using illustrative but minimal data, we've highlighted the potential use of these assets to generate visual products and give insight into the processing that takes place. 

Those interested in running forecasts with a properly scaled training and prediction set of data should now be able to consider doing so using the CLI commands. However, the CLI commands **only expose an illustrative set of commands at present**, whereas leveraging the API directly affords far more flexibility. This is explored in the remaining notebooks: 

* [Library usage](03.library_usage.ipynb): understand how to programmatically perform an end to end run.
* [Library extension](04.library_extension.ipynb): understand why and how to extend the IceNet library.

## Version
- IceNet Codebase: v0.2.5